In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import random
import numpy as np
import matplotlib.pyplot as plt

class MontyHallGame:
    def __init__(self):
        self.reset_game()
        self.stats = {'stay_wins': 0, 'stay_games': 0,
                     'switch_wins': 0, 'switch_games': 0}
        self.setup_widgets()

    def reset_game(self):
        self.doors = [0, 0, 1]  # 0 = goat, 1 = car
        random.shuffle(self.doors)
        self.selected_door = None
        self.revealed_door = None
        self.game_state = 'initial'

    def setup_widgets(self):
        # Door selection buttons
        self.door_buttons = [widgets.Button(description=f'Door {i+1}',
                                          layout=widgets.Layout(width='100px'))
                           for i in range(3)]
        for i, btn in enumerate(self.door_buttons):
            btn.on_click(lambda b, i=i: self.select_door(i))

        # Decision buttons
        self.stay_button = widgets.Button(description='Stay',
                                        layout=widgets.Layout(width='100px'))
        self.switch_button = widgets.Button(description='Switch',
                                          layout=widgets.Layout(width='100px'))
        self.stay_button.on_click(lambda b: self.make_decision(False))
        self.switch_button.on_click(lambda b: self.make_decision(True))

        # Play again button
        self.play_again_button = widgets.Button(description='Play Again',
                                              layout=widgets.Layout(width='100px'))
        self.play_again_button.on_click(lambda b: self.play_again())

        # Status output
        self.status_output = widgets.Output()
        self.stats_output = widgets.Output()

        # Layout
        self.doors_box = widgets.HBox(self.door_buttons)
        self.decision_box = widgets.HBox([self.stay_button, self.switch_button])
        self.main_box = widgets.VBox([
            self.doors_box,
            self.decision_box,
            self.play_again_button,
            self.status_output,
            self.stats_output
        ])

    def select_door(self, door_idx):
        if self.game_state != 'initial':
            return

        self.selected_door = door_idx

        # Host reveals a goat
        possible_reveals = [i for i in range(3)
                          if i != door_idx and self.doors[i] == 0]
        self.revealed_door = random.choice(possible_reveals)
        self.game_state = 'selected'

        self.update_display()

    def make_decision(self, switch):
        if self.game_state != 'selected':
            return

        final_door = self.selected_door
        if switch:
            final_door = next(i for i in range(3)
                            if i != self.selected_door and i != self.revealed_door)

        won = self.doors[final_door] == 1
        if switch:
            self.stats['switch_games'] += 1
            if won:
                self.stats['switch_wins'] += 1
        else:
            self.stats['stay_games'] += 1
            if won:
                self.stats['stay_wins'] += 1

        self.game_state = 'finished'
        self.update_display()

    def play_again(self):
        self.reset_game()
        self.update_display()

    def update_display(self):
        with self.status_output:
            clear_output(wait=True)
            self.display_game_state()

        with self.stats_output:
            clear_output(wait=True)
            self.display_stats()

    def display_game_state(self):
        if self.game_state == 'initial':
            print("Select a door!")
        elif self.game_state == 'selected':
            print(f"You selected Door {self.selected_door + 1}")
            print(f"Host revealed a goat behind Door {self.revealed_door + 1}")
            print("Would you like to stay with your choice or switch?")
        elif self.game_state == 'finished':
            print("Game Over!")
            print(f"Car was behind Door {self.doors.index(1) + 1}")

    def display_stats(self):
        stay_rate = (self.stats['stay_wins'] / self.stats['stay_games'] * 100
                    if self.stats['stay_games'] > 0 else 0)
        switch_rate = (self.stats['switch_wins'] / self.stats['switch_games'] * 100
                      if self.stats['switch_games'] > 0 else 0)

        print("\nStats:")
        print(f"Stay Strategy: {self.stats['stay_wins']}/{self.stats['stay_games']} "
              f"({stay_rate:.1f}% win rate)")
        print(f"Switch Strategy: {self.stats['switch_wins']}/{self.stats['switch_games']} "
              f"({switch_rate:.1f}% win rate)")

    def run(self):
        display(self.main_box)
        self.update_display()

# Example usage:
# game = MontyHallGame()
# game.run()


In [ ]:
game = MontyHallGame()
game.run()